In [ ]:
# 한글 폰트 저장
!sudo apt-get install -y fonts-nanum
!sudo fc-cache -fv
!rm ~/.cache/matplotlib -rf

import matplotlib.pyplot as plt
from matplotlib import rc

plt.rc('font', family='NanumGothic')
%matplotlib inline

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-nanum
0 upgraded, 1 newly installed, 0 to remove and 19 not upgraded.
Need to get 10.3 MB of archives.
After this operation, 34.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-nanum all 20200506-1 [10.3 MB]
Fetched 10.3 MB in 2s (5,478 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package fonts-nanum.
(Reading database ... 120874 files and dire

In [ ]:
# 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/DCC/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/DCC


In [ ]:
# 필요한 라이브러리 불러오기
import torch
import os
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action="ignore")

In [ ]:
# cuda 사용
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [ ]:
# 데이터셋의 class 종류 저장 - 이름: class_names
path = "kfood_health_train/"
class_names = os.listdir(path)
print(class_names)
print(len(class_names))

['돼지갈비찜', '김치국', '떡국', '된장찌개', '갈비찜', '배추김치', '낚지볶음', '모듬초밥', '가리비', '고등어', '순대', '부대찌개', '오리로스구이']
13


In [ ]:
# 필요한 라이브러리 불러오기
import os

import torch
import torchvision
from torchvision import transforms, datasets
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# 데이터 파일 경로 설정
data_root_train='kfood_health_train'
data_root_val='kfood_health_val'

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# 데이터 불러오기
data_train = datasets.ImageFolder(root=data_root_train, transform=transform)
data_val= datasets.ImageFolder(root=data_root_val, transform=transform)

In [ ]:
# 데이터 로더 설정
batch_size = 128
train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(data_val, batch_size=batch_size)

In [ ]:
# 필요한 라이브러리 불러오기
from tqdm.notebook import tqdm
from torch import nn, optim
from torch.autograd import Variable
from torchvision import models
from torch.backends import cudnn
cudnn.benchmark = True

In [ ]:
# 모델 학습 함수
def train(model, epochs, batch, criterion, optimizer):

    best_acc = 0.0

    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(f'device: {device}')

    model.to(device)

    for epoch in range(epochs):
        running_loss = 0.0
        running_acc = 0.0
        correct_train = 0
        total_train = 0

        for i, (images, labels) in enumerate(train_loader, 0):
            images = Variable(images.to(device))
            labels = Variable(labels.to(device))

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        print("-----------------")
        print(f"> Epoch {epoch+1}/{epochs}")
        running_loss = running_loss / total_train
        running_acc = 100 * correct_train / total_train
        print(f"Train Loss: {running_loss:.4f},  Train Accuracy: {running_acc:.4f}%")

        acc = validate_epoch(model, val_loader, device)
        print(f'Validation Accuracy: {acc:.4f}%')

        if acc > best_acc:
            torch.save({
                'model': model,
                'epoch': epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': running_loss
                }, "./mission3.pt")
            best_acc = acc

In [ ]:
# DCC 측에서 제공받은 코드(변형X)
@torch.no_grad()
def validate_epoch(model: nn.Module, data_loader: DataLoader, device: torch.device):
    ''' data_loader provides inputs and GTs.
        model receives input images from data_loader and produces logits.
        returns the accuracy of model in percent.
    '''
    model.eval()
    accuracies = []
    for images, labels in tqdm(data_loader, total=len(data_loader), mininterval=1, desc='measuring accuracy'):
        images = images.to(device)
        labels = labels.to(device)

        logits = model(images)
        pred = torch.argmax(logits, dim=1)

        accuracies.append(pred == labels)

    accuracy = torch.concat(accuracies).float().mean() * 100
    return accuracy.item()

In [ ]:
from torchsummary import summary

In [ ]:
# 모델 정의(ResNet34)
model3 = models.resnet34(pretrained=False)
model3.fc = nn.Linear(model3.fc.in_features, 13)
model3.to(device)

In [ ]:
# hyperparameters 설정
batch = 128
epochs = 50
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model3.parameters(), lr=0.0001)

In [ ]:
# 모델 불러오기
checkpoint = torch.load('./model2.pt')
model3.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
checkpoint_epoch = checkpoint["epoch"]
checkpoint_loss = checkpoint["loss"]

for param in model3.parameters():
    param.requires_grad = False

In [ ]:
val_accuracy = validate_epoch(model3, val_loader, device)
val_accuracy

measuring accuracy:   0%|          | 0/4 [00:00<?, ?it/s]

6.313992977142334

In [ ]:
# 클래스별 정확도를 저장할 딕셔너리 초기화
class_correct = {classname: 0 for classname in data_train.classes}
class_total = {classname: 0 for classname in data_train.classes}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model3.to(device)

# 그라디언트가 필요하지 않으므로 no_grad 컨텍스트 내에서 예측 수행
with torch.no_grad():
    for data in val_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model3(images)  # 모델을 사용하여 예측 수행
        _, predictions = torch.max(outputs, 1)

        for label, prediction in zip(labels, predictions):
            if label == prediction:
                class_correct[data_train.classes[label]] += 1
            class_total[data_train.classes[label]] += 1

# 각 클래스별 정확도 출력
sorted_accuracy = sorted(class_correct.items(), key=lambda x: x[1] / class_total[x[0]], reverse=True)

for classname, _ in sorted_accuracy:
    correct_count = class_correct[classname]
    total_count = class_total[classname]
    accuracy = 100 * float(correct_count) / total_count
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

Accuracy for class: 순대 is 69.0 %
Accuracy for class: 낚지볶음 is 13.9 %
Accuracy for class: 떡국 is 11.0 %
Accuracy for class: 배추김치 is 6.2 %
Accuracy for class: 오리로스구이 is 4.3 %
Accuracy for class: 고등어 is 1.5 %
Accuracy for class: 부대찌개 is 0.9 %
Accuracy for class: 된장찌개 is 0.9 %
Accuracy for class: 김치국 is 0.6 %
Accuracy for class: 가리비 is 0.0 %
Accuracy for class: 갈비찜 is 0.0 %
Accuracy for class: 돼지갈비찜 is 0.0 %
Accuracy for class: 모듬초밥 is 0.0 %


In [ ]:
train(model3, epochs, batch, criterion, optimizer)
print('Finished Training')

NameError: ignored

In [ ]:
val_accuracy = validate_epoch(model3, val_loader, device)
val_accuracy

NameError: ignored

In [ ]:
# 클래스별 정확도를 저장할 딕셔너리 초기화
class_correct = {classname: 0 for classname in data_train.classes}
class_total = {classname: 0 for classname in data_train.classes}
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model3.to(device)

# 그라디언트가 필요하지 않으므로 no_grad 컨텍스트 내에서 예측 수행
with torch.no_grad():
    for data in val_loader:
        images, labels = data[0].to(device), data[1].to(device)
        outputs = model3(images)  # 모델을 사용하여 예측 수행
        _, predictions = torch.max(outputs, 1)

        for label, prediction in zip(labels, predictions):
            if label == prediction:
                class_correct[data_train.classes[label]] += 1
            class_total[data_train.classes[label]] += 1

# 각 클래스별 정확도 출력
sorted_accuracy = sorted(class_correct.items(), key=lambda x: x[1] / class_total[x[0]], reverse=True)

for classname, _ in sorted_accuracy:
    correct_count = class_correct[classname]
    total_count = class_total[classname]
    accuracy = 100 * float(correct_count) / total_count
    print(f'Accuracy for class: {classname:5s} is {accuracy:.1f} %')

NameError: ignored

In [ ]:
# 결과 시각화 함수
def showResult(model, batch):
    images, labels = next(iter(val_loader))

    img = torchvision.utils.make_grid(images)
    img = img / 2 + 0.5
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

    print('Real class: ', ' '.join('%5s' % class_names[labels[j]] for j in range(batch)))


    outputs = model(images)
    _, predicted = torch.max(outputs, 1)

    print('Predicted class: ', ' '.join('%5s' % class_names[predicted[j]] for j in range(batch)))

In [ ]:
showResult(model3, batch)